# 1. 터미널을 통해 스파크 서버의 접속 정보를 확인합니다
```bash
bash> cd ~/workspace/data-engineer-basic-training/day5
bash> docker-compose up -d
bash> docker-compose ps

# 아래와 같이 http://127.0.0.1 로 시작하는 URL 에서 127.0.0.1 부분을 현재 aws-instance-host 이름으로 변경합니다
bash> docker-compose logs notebook | grep http
...
or http://127.0.0.1:8888/?token=ad4f43203ac46f7f7f58807ab6781b1fd18b9ca5066664df
...
```

### 1-1. 크롬을 통해서 [http://aws-hostname:8888/?token=...](http://<aws-hostname>:8888/?token=ad4f43203ac46f7f7f58807ab6781b1fd18b9ca5066664df) 에 접속합니다

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("Data Engineer Basic Day5") \
    .config("spark.dataengineer.basic.day5", "step4") \
    .config("spark.sql.session.timeZone", "Asia/Seoul") \
    .getOrCreate()

# 2. 필요한 모든 데이터를 읽어서 데이터프레임으로 생성합니다

### 2-1. 각 경로에서 10/26 기준 데이터를 읽고 스키마와 데이터를 확인합니다

In [ ]:
# 1. 오늘은 2020/10/27 이고, 어제가 서비스 둘째 날인 2020/10/26 날짜의 기준 지표를 생성합니다

user26 = spark.read.parquet("user/20201026")
user26.createOrReplaceTempView("user26") # 테이블생성
user26.printSchema() # 스키마 출력
user26.show(truncate=False) # 데이터 확인

purchase26 = spark.read.parquet("purchase/20201026")
purchase26.createOrReplaceTempView("purchase26")
purchase26.printSchema()
purchase26.show(truncate=False)


accesslog = spark.read.option("inferSchema", "true").json("access/20201026")
accesslog.createOrReplaceTempView("accesslog")
access26 = spark.sql("select a_id, a_tag, a_time, a_timestamp, a_uid from accesslog")
access26.createOrReplaceTempView("access26")
access26.printSchema()
access26.show(truncate=False)


### 2-2. 사용에 용이하게 하기 위해 2020/10/26 일자의 데이터만 필터하여 데이터프레임을 생성합니다
> unixtime 을 문자열 시간으로 변경하는 함수는 from_unixtime 입니다

In [ ]:
p_time_condition = "p_time >= '2020-10-26 00:00:00' and p_time < '2020-10-27 00:00:00'"

purchase = spark.sql("select from_unixtime(p_time) as p_time, p_uid, p_id, p_name, p_amount from purchase26").where(p_time_condition)
purchase.createOrReplaceTempView("purchase")
purchase.printSchema()
purchase.show(truncate=False)

u_signup_condition = "u_signup >= '20201026' and u_signup < '20201027'"

user = spark.sql("select u_id, u_name, u_gender from user26").where(u_signup_condition)
user.createOrReplaceTempView("user")
user.printSchema()
user.show(truncate=False)

access = spark.sql("select a_id, a_tag, a_timestamp, a_uid from access26")
access.createOrReplaceTempView("access")
access.printSchema()
access.show(truncate=False)

# 3. Daily Activer User 지표를 생성하세요

### 3-1. access 테이블을 이용하여 DAU 지표를 생성하세요
```text
지표정의 : 지정한 일자의 접속한 유저 수
지표산식 : 지정한 일자의 접속 테이블에 로그(로그인 혹은 로그아웃)가 한 번 이상 발생한 이용자의 빈도수
입력형태 : access 테이블
출력형태 : number
```

In [ ]:
spark.sql("select a_timestamp as a_time, a_uid from access").show(truncate=False)

distinctAccessUser = "select count(distinct a_uid) as DAU from access"

dau = spark.sql(distinctAccessUser)
dau.show()

# 4. Daily Paying User 지표를 생성하세요

### 4-1. purchase 테이블을 이용하여 PU 지표를 생성하세요
```text
지표정의 : 지정한 일자의 구매 유저 수
지표산식 : 지정한 일자의 구매 테이블에 한 번이라도 구매가 발생한 이용자의 빈도수
입력형태 : purchase 테이블
출력형태 : number
```

In [ ]:
spark.sql("select p_time, p_uid from purchase").show()

distinctPayingUser = "select count(distinct p_uid) as PU from purchase" 

pu = spark.sql(distinctPayingUser)
pu.show()

# 5. Daily Revenue 지표를 생성하세요

### 5-1. purchase 테이블을 이용하여 DR 지표를 생성하세요
```text
지표정의 : 지정한 일자에 발생한 총 매출 금액
지표산식 : 지정한 일자의 구매 테이블에 저장된 전체 매출 금액의 합
입력형태 : access 테이블
출력형태 : number
```

In [ ]:
spark.sql("select p_amount as daily_revenue from purchase").show()

sumOfDailyRevenue = "select sum(p_amount) as DR from purchase" 

dr = spark.sql(sumOfDailyRevenue)
dr.show()

# 6. ARPU - Average Revenue Per User 지표를 생성하세요
> 이미 생성된 데이터프레임으로부터 이름으로 하나의 값을 가져오는 방법은 아래와 같습니다
```python
변수명 = 데이터프레임.collect()[0]["이름"]
v_dau = dau.collect()[0]["DAU"]
```


### 5-1. daily revenue 와 daily user 정보를 이용하여 ARPU 를 구하세요
```text
지표정의 : 유저 당 평균 발생 매출 금액
지표산식 : 총 매출 / 전체 유저 수 = DR / DAU
입력형태 : Daily Revenue, Daily Active User
출력형태 : number
```

In [ ]:
v_dau = dau.collect()[0]["DAU"]
v_pu = pu.collect()[0]["PU"]
v_dr = dr.collect()[0]["DR"]

print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_dau))
print("+------------------+")

# 6. ARPPU - Average Revenue Per Paying User 지표를 생성하세요
> 이미 생성된 데이터프레임으로부터 이름으로 하나의 값을 가져오는 방법은 아래와 같습니다
```python
변수명 = 데이터프레임.collect()[0]["이름"]
v_dau = dau.collect()[0]["DAU"]
```

### 6-1. daily revenue 와 daily paying user 정보를 이용하여 ARPU 를 구하세요
```text
지표정의 : 유저 당 평균 발생 매출 금액
지표산식 : 총 매출 / 전체 유저 수 = DR / DPU
입력형태 : Daily Revenue, Daily Paying User
출력형태 : number
```

In [ ]:
# 7. Average Revenue Per Paying User

print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_pu))
print("+------------------+")

# 7. 디멘젼 테이블을 생성하세요
> 디멘젼 테이블을 스키마는 아래와 같습니다.

| 컬럼명 | 컬럼타입 | 설명 |
| - | - | - |
| d_uid | integer | 아이디 |
| d_name | string | 이름 |
| d_pamount | integer | 누적 구매 금액 |
| d_pcount | integer | 누적 구매 횟수 |
| d_acount | integer | 누적 접속 횟수 |
| d_first_purchase | string | 최초 구매 일시 |
| dt | string | 유저아이디 |

### 7-1. 서비스 둘째 날이므로 어제의 디멘젼 테이블을 읽어옵니다
```text
지표정의 : 이용자 누적 상태 정보
지표산식 : 오늘까지 접속한 모든 유저의 정보를 저장하는 테이블
입력형태 : user, purchase, access
출력형태 : 아래와 같이 설계합니다
```

In [ ]:
yesterday_dimension="dim_users/dt=20201025"
yesterday = spark.read.parquet(yesterday_dimension)
yesterday.printSchema()
yesterday.sort(yesterday["d_uid"].asc()).show()

### 7-2. 오늘 추가된 새로운 모수를 추가해야 하므로 전체 모수에 해당하는 uid 만을 추출합니다

In [ ]:
yesterday_uids = yesterday.select("d_uid")
today_uids = access.select("a_uid")

# 실습22) joinCondition 는 어제 uid 와 오늘 uid 값이 일치하는 조건
joinCondition = ""

# 실습23) 양쪽 테이블이 모두 존재해야 하는 조인 방식
joinHow = ""

uid = yesterday_uids.join(today_uids, joinCondition, joinHow) \
.withColumn("uid", when(yesterday.d_uid.isNull(), access.a_uid).otherwise(yesterday.d_uid)) \
.select("uid").distinct()

uid.printSchema()
uid.sort("uid").show()

### 7-3. 어제 디멘젼에 과거정보를 오늘 디멘젼에 붙입니다 - d_name, d_gender, d_acount, d_pamount, d_pcount, d_first_purchase

In [ ]:
# 실습24) uid 와 yesterday 의 uid 가 일치하는 조건
joinCondition = ""
joinHow = "left_outer"

# uid 에서 생성된 id 값이 모든 user_id 이므로
# drop 함수를 통해 기존의 d_uid 는 제거하고
# withColumnRenamed 함수를 통해, uid 를 d_uid 로 변경 합니다
uids = uid.join(yesterday, joinCondition, joinHow) \
.drop("d_uid") \
.withColumnRenamed("uid", "d_uid") \
.sort("d_uid")

uids.printSchema()
uids.show(truncate=False)

### 7-4. uid 기준으로 이름, 성별을 조인합니다
```bash
expr("case when column1 is null then column2 else column1 end") 구문을 사용합니다.
```

In [ ]:
# 실습25) d_name 이 null 이면 u_name 을 사용하고 그렇지 않으면 d_name 을 사용합니다
exprName = expr("d_name")

# 실습26) d_gender 이 null 이면 u_gender 을 사용하고 그렇지 않으면 d_gender 을 사용합니다
exprGender = expr("d_gender")

dim1 = uids.join(user, uids.d_uid == user.u_id, "left_outer") \
.withColumn("name", exprName) \
.withColumn("gender", exprGender) \
.drop("d_name", "d_gender", "u_id", "u_name", "u_gender") \
.withColumnRenamed("name", "d_name").withColumnRenamed("gender", "d_gender")

dim1.show()

### 7-5. 숫자 값에 대한 널값을 기본값으로 채웁니다

In [ ]:

# 실습27) d_acount, d_pamount, d_pcount 필드의 기본값을 0으로 넣도록 작성합니다
fillDefaultValue = {}

dim2 = dim1.na.fill(fillDefaultValue)
dim2.show(truncate=False)

### 7-6. 오늘 생성된 접속수치, 매출 및 매출 횟수를 더합니다

In [ ]:
access_sum = spark.sql("select a_uid, count(a_uid) as a_count from access where a_id = 'login' group by a_uid")
access_sum.printSchema()
access_sum.show()

# 실습28) 오늘 접속 수치(a_count)가 null 이면 디멘젼의 d_acount 값을 사용하고, 그렇지 않으면 d_acount + a_count 를 사용합니다
sumOfAccess = exp("a_count")

dim3 = dim2.join(access_sum, dim2.d_uid == access_sum.a_uid, "left") \
.withColumn("sum_of_access", sumOfAccess) \
.drop("a_uid", "a_count", "d_acount") \
.withColumnRenamed("sum_of_access", "d_acount")
dim3.printSchema()
dim3.show()

### 7-7. 오늘 발생한 매출을 더합니다

In [ ]:
purchase_sum = spark.sql("select p_uid, sum(p_amount) as pamount, count(p_uid) as pcount from purchase group by p_uid")
purchase_sum.show()

# 실습29) 오늘 발생한 매출(pamount)이 null 이면 디멘젼 매출(d_pamount)를 그렇지 않으면 둘의 합(d_pamount + pamount)을 구합니다
sumOfAmount = expr("d_pamount")

# 실습30) 오늘 발생한 매출빈도(pcount)가 null 이면 디멘젼 매출빈도(d_pcount)를 그렇지 않으면 둘의 합(d_pcount + pcount)을 구합니다
sumOfCount = expr("pcount")

dim4 = dim3.join(purchase_sum, dim3.d_uid == purchase_sum.p_uid, "left") \
.withColumn("sum_of_amount", sumOfAmount) \
.withColumn("sum_of_count", sumOfCount) \
.drop("p_uid", "pamount", "pcount", "d_pamount", "d_pcount") \
.withColumnRenamed("sum_of_amount", "d_pamount") \
.withColumnRenamed("sum_of_count", "d_pcmount") \

dim4.printSchema()
dim4.show()

### 7-8. 최초 구매 일자를 업데이트합니다

In [ ]:
# 실습31) 하루에 여러번 구매가 있을 수 있으므로 group by p_uid 집계를 통해 가장 먼저 구매한 정보 즉, min(p_time)함수를 통해 일시를 선택합니다
selectFirstPurchase = ""

first_purchase = spark.sql(selectFirstPurchase)
first_purchase.printSchema()
first_purchase.show()

# 실습32) 디멘젼의 최초구매일(d_first_purchase)이 null 이라면 p_time 을 사용하고 그렇지 않으면 d_first_purchase 를 사용합니다
exprFirstPurchase = expr("d_first_purchase")

today = dim4.join(first_purchase, dim4.d_uid == first_purchase.p_uid, "left") \
.withColumn("first_purchase", exprFirstPurchase) \
.drop("p_uid", "p_time", "d_first_purchase") \
.withColumnRenamed("first_purchase", "d_first_purchase")

today.printSchema()
today.show()

### 7-9. 어제와 오늘의 디멘젼 테이블을 이용해 오늘 처음 접속한 신규 유저를 구합니다

In [ ]:
today_uids = today.select("d_uid")
yesterday_uids = yesterday.select("d_uid")

nu = today_uids.subtract(yesterday_uids)
nu.printSchema()
nu.show()
v_nu = nu.count()

# Daily Revenue Per Paying User
print("+------------------+")
print("|               NU |")
print("+------------------+")
print("|                {} |".format(v_nu))
print("+------------------+")

### 7-10. 오늘 디멘젼 테이블을 dim_users/dt=20201026 경로에 저장합니다.

In [ ]:
today_dimension = "dim_users/dt=20201026"

today.write.mode("overwrite").parquet(today_dimension)

### 7-11. 오늘의 주요 지표를 출력합니다

In [ ]:
print("+------------------+")
print("|              DAU |")
print("+------------------+")
print("|                {} |".format(v_dau))
print("+------------------+")
print("+------------------+")
print("|               PU |")
print("+------------------+")
print("|                {} |".format(v_pu))
print("+------------------+")
print("+------------------+")
print("|                DR |")
print("+------------------+")
print("|        {} |".format(v_dr))
print("+------------------+")
print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_dau))
print("+------------------+")
print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_pu))
print("+------------------+")